In [1]:
import pandas as pd 
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
import numpy as np 
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('gen_data.csv')

In [3]:
segmentation_candidates = [
    # --- NHÓM 1: QUY MÔ & TÀI SẢN (Wealth & Scale) - "Giàu hay Nghèo?" ---
    'BASE_AUM',             # Tổng tài sản (Quan trọng nhất)
    'INCOME',               # Thu nhập
    'COLLATERAL_VALUE',     # Giá trị TSBĐ (Phân biệt vay thế chấp/tín chấp)
    'SOHUUNHA',             # Có nhà hay không
    
    # --- NHÓM 2: QUY MÔ TÍN DỤNG (Exposure) - "Vay nhiều hay ít?" ---
    'AFLIMT_MAX',           # Hạn mức (Sức chứa)
    'CBAL',                 # Dư nợ hiện tại
    'CBAL_MAX',             # Đỉnh nợ
    'LTV',
    
    # --- NHÓM 3: SẢN PHẨM & MỤC ĐÍCH (Product) - "Vay để làm gì?" ---
    # (MỚI BỔ SUNG - Rất quan trọng để tách Portfolio)
    'PURCOD_MAX',           # Mã mục đích vay (BĐS, Tiêu dùng, SXKD...)
    'HAS_SHORTTERM_LOAN',   # Chỉ vay ngắn hạn?
    'HAS_LONGTERM_LOAN',    # Có vay dài hạn?
    'DURATION_MAX',         # Kỳ hạn vay (Ngắn/Trung/Dài)
    
    # --- NHÓM 4: THANH KHOẢN (Liquidity) - "Có tiền mặt không?" ---
    'N_AVG_DEPOSIT_12M',    # Tiền gửi bình quân
    'FLAG_SALARY_ACC',      # Có trả lương qua đây không?
    'FLAG_DEPOSIT',         # Có tài khoản tiền gửi không?
    
    # --- NHÓM 5: NHÂN KHẨU (Demographics) - "Ai?" ---
    'TUOI',                 # Độ tuổi (Người trẻ rủi ro khác người già)
    'TRINHDO',              # Học vấn
    'TTHONNHAN'             # Hôn nhân
]

X = df[segmentation_candidates].copy()
y = df['BAD_NEXT_12M'].copy()

# Xử lý biến phân loại (Categorical)
# Tự động tìm các cột dạng chuỗi (Object) hoặc Category
categorical_cols = X.select_dtypes(include=['object', 'category', 'bool']).columns

for col in categorical_cols:
    # Bước 1: Điền giá trị thiếu cho biến chữ trước (nếu có)
    X[col] = X[col].fillna("Unknown")
    
    # Bước 2: Chuyển sang dạng số (0, 1, 2...)
    # Ví dụ: Single -> 0, Married -> 1
    X[col] = pd.factorize(X[col])[0]
    print(f" -> Đã mã hóa biến: {col}")
    
# Điền giá trị 0 vào các ô trống (hoặc dùng trung bình tùy nghiệp vụ)
imputer = SimpleImputer(strategy='constant', fill_value=0)                       # ko có null

X_filled = imputer.fit_transform(X)

# 4. Cấu hình Cây quyết định (Theo đúng Báo cáo BIDV)
# criterion='gini', class_weight='balanced'
dt_model = DecisionTreeClassifier(
    criterion='gini',
    splitter='best',
    class_weight='balanced',
    random_state=42,
    max_depth=5  # Giới hạn độ sâu để tránh overfitting khi tính độ quan trọng
)

# Huấn luyện mô hình
dt_model.fit(X_filled, y)

# 5. Trích xuất Feature Importance
importance_df = pd.DataFrame({
    'Tên Biến': segmentation_candidates,
    'Độ Quan Trọng (Feature Importance)': dt_model.feature_importances_
})

# Sắp xếp từ cao xuống thấp
importance_df = importance_df.sort_values(by='Độ Quan Trọng (Feature Importance)', ascending=False)

print("--- KẾT QUẢ XẾP HẠNG ĐỘ QUAN TRỌNG CỦA BIẾN ---")
print(importance_df)

 -> Đã mã hóa biến: TTHONNHAN
--- KẾT QUẢ XẾP HẠNG ĐỘ QUAN TRỌNG CỦA BIẾN ---
              Tên Biến  Độ Quan Trọng (Feature Importance)
2     COLLATERAL_VALUE                            0.651133
12   N_AVG_DEPOSIT_12M                            0.312930
7                  LTV                            0.023635
5                 CBAL                            0.008419
0             BASE_AUM                            0.002894
6             CBAL_MAX                            0.000989
11        DURATION_MAX                            0.000000
16             TRINHDO                            0.000000
15                TUOI                            0.000000
14        FLAG_DEPOSIT                            0.000000
13     FLAG_SALARY_ACC                            0.000000
9   HAS_SHORTTERM_LOAN                            0.000000
10   HAS_LONGTERM_LOAN                            0.000000
1               INCOME                            0.000000
8           PURCOD_MAX               

In [4]:
# from sklearn.tree import plot_tree
# import matplotlib.pyplot as plt

# # Cấu hình lại cây nông hơn (max_depth=3) để dễ nhìn biểu đồ
# dt_viz = DecisionTreeClassifier(
#     criterion='gini', 
#     max_depth=3,            # Chỉ vẽ 3 tầng
#     min_samples_leaf=0.05, 
#     class_weight='balanced',
#     random_state=42
# )
# dt_viz.fit(X_filled, y)

# # Vẽ hình
# plt.figure(figsize=(25, 12))
# plot_tree(
#     dt_viz, 
#     feature_names=segmentation_candidates, 
#     class_names=['Good', 'Bad'],
#     filled=True, 
#     rounded=True, 
#     fontsize=12,
#     impurity=False, # Ẩn chỉ số Gini cho đỡ rối
#     proportion=True # Hiển thị tỷ lệ %
# )
# plt.title("SƠ ĐỒ PHÂN KHÚC KHÁCH HÀNG (Dựa trên Collateral & Deposit)")
# plt.show()

In [5]:
# ==========================================
# 11. TÌM NGƯỠNG CẮT TỐI ƯU (OPTIMAL BINNING)
# ==========================================
from optbinning import OptimalBinning
import pandas as pd

print("\n=== TÌM NGƯỠNG CẮT TỐI ƯU (DATA-DRIVEN THRESHOLDS) ===")

# Hàm hỗ trợ tìm điểm cắt
def find_optimal_cutoff(df, feature, target):
    # Cấu hình Optimal Binning
    # max_n_bins=2 -> Chỉ chia làm 2 phần (<= Ngưỡng và > Ngưỡng)
    optb = OptimalBinning(name=feature, dtype="numerical", solver="cp", max_n_bins=2)
    
    # Fit dữ liệu
    optb.fit(df[feature], df[target])
    
    # Lấy bảng kết quả binning
    binning_table = optb.binning_table.build()
    
    # Lấy điểm cắt (Split point)
    # Thường nó nằm ở dòng đầu tiên cột 'Bin' (dạng [-inf, 10000.0))
    splits = optb.splits
    
    print(f"\n--- Phân tích biến: {feature} ---")
    if len(splits) > 0:
        cutoff = splits[0]
        print(f"-> Điểm cắt tối ưu (Math): {cutoff:,.0f}")
        print(f"-> Gợi ý làm tròn (Business): {round(cutoff, -6):,.0f}") # Làm tròn đến hàng triệu
        print("Chi tiết chia nhóm:")
        display(binning_table[['Bin', 'Count', 'Event', 'Non-event', 'Event rate', 'IV']])
        return cutoff
    else:
        print("-> Không tìm thấy điểm cắt tối ưu (Biến này không phân tách được rủi ro).")
        return None

# 1. Tìm ngưỡng cho TÀI SẢN ĐẢM BẢO (Collateral)
# Để tách nhóm Secured vs Unsecured
cutoff_collateral = find_optimal_cutoff(df, 'COLLATERAL_VALUE', 'BAD_NEXT_12M')

# 2. Tìm ngưỡng cho TIỀN GỬI (Deposit)
# Để tách nhóm VIP vs Mass (Chỉ chạy trên tập KHÔNG có TSBĐ để chuẩn bài)
# Vì nhóm có TSBĐ đã tách ra rồi, ta chỉ quan tâm phân khúc VIP trong nhóm Tín chấp
df_unsecured = df[df['COLLATERAL_VALUE'] <= (cutoff_collateral if cutoff_collateral else 0)]
cutoff_deposit = find_optimal_cutoff(df_unsecured, 'N_AVG_DEPOSIT_12M', 'BAD_NEXT_12M')

# ==========================================
# CẬP NHẬT LẠI BIẾN CẮT (Nếu muốn dùng số máy tính)
# ==========================================
# THRESH_COLLATERAL = cutoff_collateral
# THRESH_DEPOSIT = cutoff_deposit


=== TÌM NGƯỠNG CẮT TỐI ƯU (DATA-DRIVEN THRESHOLDS) ===

--- Phân tích biến: COLLATERAL_VALUE ---
-> Điểm cắt tối ưu (Math): 36,214,774
-> Gợi ý làm tròn (Business): 36,000,000
Chi tiết chia nhóm:


,Bin,Count,Event,Non-event,Event rate,IV
0,"(-inf, 36214774.00)",681516,133157,548359,0.195384,0.237670
1,"[36214774.00, inf)",456643,4563,452080,0.009992,1.094153
2,Special,0,0,0,0.000000,0.000000
3,Missing,0,0,0,0.000000,0.000000
Totals,,1138159,137720,1000439,0.121002,1.331822



--- Phân tích biến: N_AVG_DEPOSIT_12M ---
-> Điểm cắt tối ưu (Math): 3,273,152
-> Gợi ý làm tròn (Business): 3,000,000
Chi tiết chia nhóm:


,Bin,Count,Event,Non-event,Event rate,IV
0,"(-inf, 3273152.50)",420200,115541,304659,0.274967,0.139154
1,"[3273152.50, inf)",261316,17616,243700,0.067413,0.378207
2,Special,0,0,0,0.000000,0.000000
3,Missing,0,0,0,0.000000,0.000000
Totals,,681516,133157,548359,0.195384,0.517362


In [6]:
# ==========================================
# 12. CHỐT PHÂN KHÚC (FINAL SEGMENTATION)
# ==========================================
print("\n=== ÁP DỤNG NGƯỠNG CẮT ĐỂ CHIA 3 PHÂN KHÚC ===")

# 1. Định nghĩa ngưỡng (Làm tròn từ kết quả Optimal Binning)
THRESH_COLLATERAL = 2_000_000_000  # 2 Tỷ VND
THRESH_DEPOSIT    = 3_500_000      # 3.5 Triệu VND

print(f"-> Ngưỡng TSBĐ: {THRESH_COLLATERAL:,.0f} VND")
print(f"-> Ngưỡng Tiền gửi: {THRESH_DEPOSIT:,.0f} VND")

# 2. Gán nhãn Phân khúc
conditions = [
    # Nhóm 1: RICH / SECURED (Có TSBĐ lớn hơn 2 tỷ) -> Rủi ro cực thấp (0.6%)
    (df['COLLATERAL_VALUE'] >= THRESH_COLLATERAL),
    
    # Nhóm 2: PRIME UNSECURED (Không TSBĐ lớn, nhưng có tiền gửi > 3.5tr) -> Rủi ro vừa (6%)
    (df['COLLATERAL_VALUE'] < THRESH_COLLATERAL) & (df['N_AVG_DEPOSIT_12M'] >= THRESH_DEPOSIT),
    
    # Nhóm 3: SUBPRIME / MASS (Không TSBĐ lớn, tiền gửi ít < 3.5tr) -> Rủi ro cao (26%)
    (df['COLLATERAL_VALUE'] < THRESH_COLLATERAL) & (df['N_AVG_DEPOSIT_12M'] < THRESH_DEPOSIT)
]

choices = ['1. Rich/Secured', '2. Prime Unsecured', '3. Mass Unsecured']

df['SEGMENT'] = np.select(conditions, choices, default='Unknown')

# 3. Kiểm tra kết quả lần cuối
print("\n--- HIỆU QUẢ PHÂN KHÚC (SEGMENT PERFORMANCE) ---")
summary = df.groupby('SEGMENT').agg({
    'SOCIF': 'count',
    'BAD_NEXT_12M': 'mean',
    'CBAL': 'mean',
    'INCOME': 'mean',
    'COLLATERAL_VALUE': 'mean'
}).rename(columns={
    'SOCIF': 'Số lượng KH',
    'BAD_NEXT_12M': 'Bad Rate',
    'CBAL': 'Dư nợ TB',
    'INCOME': 'Thu nhập TB',
    'COLLATERAL_VALUE': 'TSBĐ TB'
})

pd.options.display.float_format = '{:,.2f}'.format
print(summary)

# 4. Tách DataFrame để chuẩn bị cho bước Modeling tiếp theo
df_seg1 = df[df['SEGMENT'] == '1. Rich/Secured'].copy()
df_seg2 = df[df['SEGMENT'] == '2. Prime Unsecured'].copy()
df_seg3 = df[df['SEGMENT'] == '3. Mass Unsecured'].copy()


=== ÁP DỤNG NGƯỠNG CẮT ĐỂ CHIA 3 PHÂN KHÚC ===
-> Ngưỡng TSBĐ: 2,000,000,000 VND
-> Ngưỡng Tiền gửi: 3,500,000 VND

--- HIỆU QUẢ PHÂN KHÚC (SEGMENT PERFORMANCE) ---
                    Số lượng KH  Bad Rate       Dư nợ TB   Thu nhập TB  \
SEGMENT                                                                  
1. Rich/Secured          421292      0.01 370,931,597.14 25,443,505.53   
2. Prime Unsecured       279495      0.06 374,181,368.54 26,571,712.17   
3. Mass Unsecured        437372      0.27 331,460,037.04 22,643,870.90   

                            TSBĐ TB  
SEGMENT                              
1. Rich/Secured    2,014,876,589.99  
2. Prime Unsecured    20,533,948.95  
3. Mass Unsecured      4,948,737.72  


In [7]:
df_seg1.to_csv('seg1.csv', index=False)
df_seg2.to_csv('seg2.csv', index=False)
df_seg3.to_csv('seg3.csv', index=False)

In [8]:
# ==========================================
# 13. TÍNH IV (INFORMATION VALUE) CHO TỪNG PHÂN KHÚC
# ==========================================
import pandas as pd
import numpy as np

def calculate_iv(df, feature, target):
    # Chia bin đơn giản (10 bins) để tính IV nhanh
    try:
        df_bin = pd.DataFrame()
        df_bin['bin'] = pd.qcut(df[feature], q=10, duplicates='drop')
        df_bin['y'] = df[target]
        
        grp = df_bin.groupby('bin')['y'].agg(['count', 'sum'])
        grp['non_event'] = grp['count'] - grp['sum']
        
        # Tránh chia cho 0
        grp['pct_event'] = (grp['sum'] + 0.5) / (grp['sum'].sum() + 0.5)
        grp['pct_non_event'] = (grp['non_event'] + 0.5) / (grp['non_event'].sum() + 0.5)
        
        grp['woe'] = np.log(grp['pct_non_event'] / grp['pct_event'])
        grp['iv'] = (grp['pct_non_event'] - grp['pct_event']) * grp['woe']
        
        return grp['iv'].sum()
    except:
        return 0

exclude_cols = [
    'SOCIF', 'SEGMENT', 'SAMPLE_TYPE', 'year', 'BAD_NEXT_12M', 
    'BAD_CURRENT', 'SNAPSHOT_DATE', 'BAD_EVENT_DATE', 'CURE_DATE',
    'XULYNO', 'MAX_NHOMNOCIC' # Biến này tương đương target, nên bỏ ra
]

# Lấy tất cả cột số
all_numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
features_to_check = [col for col in all_numeric_cols if col not in exclude_cols]

print(f"Tổng số biến sẽ quét IV: {len(features_to_check)} biến")

# Chạy vòng lặp cho 3 phân khúc
segments = ['1. Rich/Secured', '2. Prime Unsecured', '3. Mass Unsecured']
results = {}

print("\n=== KẾT QUẢ IV (SẮP XẾP TỪ CAO XUỐNG THẤP) ===")

for seg in segments:
    df_sub = df[df['SEGMENT'] == seg]
    iv_scores = {}
    
    for feat in features_to_check:
        iv = calculate_iv(df_sub, feat, 'BAD_NEXT_12M')
        # Chỉ lấy biến có IV > 0.02 (Có chút ý nghĩa trở lên)
        if iv > 0.02:
            iv_scores[feat] = iv
    
    # Sắp xếp giảm dần
    top_features = sorted(iv_scores.items(), key=lambda x: x[1], reverse=True)
    
    print(f"\n🔹 PHÂN KHÚC: {seg} (Top 10 biến mạnh nhất)")
    for feat, iv in top_features[:10]: # Xem top 10
        strength = "Mạnh" if iv > 0.3 else ("Trung bình" if iv > 0.1 else "Yếu")
        print(f"   - {feat:25} : IV = {iv:.4f} ({strength})")

Tổng số biến sẽ quét IV: 43 biến

=== KẾT QUẢ IV (SẮP XẾP TỪ CAO XUỐNG THẤP) ===

🔹 PHÂN KHÚC: 1. Rich/Secured (Top 10 biến mạnh nhất)
   - MAX_DPD_12M               : IV = 4.6510 (Mạnh)
   - MAX_DPD_12M_OBS           : IV = 4.6510 (Mạnh)
   - SUM_ALL_OD_12M            : IV = 4.0638 (Mạnh)
   - AVG_OD_DPD_12M            : IV = 3.5730 (Mạnh)
   - N_AVG_DEPOSIT_12M         : IV = 1.5116 (Mạnh)
   - N_AVG_DEPOSIT_6M          : IV = 1.5116 (Mạnh)
   - N_AVG_DD_12M              : IV = 1.5116 (Mạnh)
   - N_AVG_CD_12M              : IV = 1.5116 (Mạnh)
   - N_AVG_OVERDUE_CBAL_12M    : IV = 0.5111 (Mạnh)
   - BASE_AUM                  : IV = 0.4895 (Mạnh)

🔹 PHÂN KHÚC: 2. Prime Unsecured (Top 10 biến mạnh nhất)
   - MAX_DPD_12M               : IV = 3.0615 (Mạnh)
   - MAX_DPD_12M_OBS           : IV = 3.0615 (Mạnh)
   - SUM_ALL_OD_12M            : IV = 2.8162 (Mạnh)
   - AVG_OD_DPD_12M            : IV = 2.5652 (Mạnh)
   - CURRENT_RISK              : IV = 0.8822 (Mạnh)
   - N_AVG_DEPOSIT_12M      

In [9]:
df.to_csv("gen_data.csv", index=False)